# Corpusanalyse UiTdatabank

We hebben de volgende bibliotheken nodig:

In [1]:
from pandas import read_excel, read_csv, DataFrame, Series, concat
from datetime import datetime
from codecs import open
from re import compile, IGNORECASE
from json import dumps
from datetime import datetime

Dan lezen we lezen de gegevens in

In [2]:
df_podium = read_excel("ruwe data/podium.xlsx", sheetname='theaterdans1014')
df_bk = read_excel("ruwe data/beeldendekunsten.xlsx", sheetname='UitRapport')
df_muziek = read_excel("ruwe data/muziek.xlsx", sheetname='Int nat reg')

Voor de muziekgegevens moeten we nog controleren dat concerten eventuele herhalingen hebben, door in de speelmomenten kolom na te gaan wat de speelmomenten zijn. Dit is niet nodig bij de beeldende kunsten, aangezien daar de unit of analysis de tentoonstelling is, onafhankelijk van hoelang die tentoonstelling loopt. Bij podiumkunsten zijn de speelmomenten manueel gecheckt, ocharme simon.

In [3]:
datumregex = compile(r"\d\d/\d\d/\d\d")
df_muziek_expanded = df_muziek.copy()
for row in df_muziek_expanded.iterrows():
    speelmomenten = row[1]["Speelmomenten"]
    if str(speelmomenten) != "nan":    
        for speelmoment in datumregex.findall(speelmomenten):
            speelmoment_dt = datetime(int("20" + speelmoment.split("/")[2]), 
                                      int(speelmoment.split("/")[1]), 
                                      int(speelmoment.split("/")[0]))
            if speelmoment_dt != row[1]["Datum"]:
                df_muziek_expanded = df_muziek_expanded.append(
                    Series(
                        {"Discipline": row[1]["Discipline"],
                         "Subdiscipline": row[1]["Subdiscipline"],
                         "Tekst": row[1]["Tekst"],
                         "Datum": speelmoment_dt,
                         "Gemeente": row[1]["Gemeente"]
                        }, name=speelmoment_dt.isoformat() + " " + str(row[0])
                    )
                )

We perken de gegevens in tot 2014, en gooien ook duplicaten op basis van datum, gemeente en tekst eruit. Bovendien hebben we de kolom met speelmomenten ook niet meer nodig.

In [4]:
df_muziek_expanded_geenspeelmomenten = df_muziek_expanded.drop("Speelmomenten", axis=1)
df = concat([df_podium, df_bk, df_muziek_expanded_geenspeelmomenten])
df = df[df["Datum"].between(datetime(2014, 1, 1), datetime(2014, 12, 31))]
df.drop_duplicates(subset=["Datum", "Discipline", "Subdiscipline", "Gemeente", "Tekst"], inplace=True)

We kunnen kort inspecteren hoe deze data eruitzien.

In [5]:
df.head()

,Datum,Discipline,Gemeente,Subdiscipline,Tekst,Unnamed: 4,Unnamed: 6
747,2014-04-02,podium,Brussel,Dansvoorstelling,Shen Yun brengt 5.000 jaar Chinese beschaving ...,NaN,NaN
748,2014-04-03,podium,Brussel,Dansvoorstelling,Shen Yun brengt 5.000 jaar Chinese beschaving ...,NaN,NaN
749,2014-04-04,podium,Brussel,Dansvoorstelling,Shen Yun brengt 5.000 jaar Chinese beschaving ...,NaN,NaN
750,2014-04-05,podium,Brussel,Dansvoorstelling,Shen Yun brengt 5.000 jaar Chinese beschaving ...,NaN,NaN
752,2014-04-06,podium,Brussel,Dansvoorstelling,Shen Yun brengt 5.000 jaar Chinese beschaving ...,NaN,NaN


We zien dat elke lijn een event beschrijving bevat, de plaats waar een event plaatsvindt, en ook de datum. Merk op dat events die op meerdere dagen plaatsvinden een aparte lijn krijgen. We zullen hiermee rekening houden in de interpretatie van de resultaten.

Frequentielijst van woorden met een hoofdletter die na in of uit voorkomen, of na een lidwoord.

Voor onze analyse hebben we ook nood aan een lijst van namen van landen, coordinaten voor de plaatsnamen, en ook een manueel gemaakte mapping om de plaatsnamen in de UiTdatabank gegevens te normaliseren.

In [6]:
typering = read_csv("extra gegevens/mapping_udb-gemeente_fusie-gemeente.csv", delimiter=';')
coord = read_csv("extra gegevens/coordinaten.csv", delimiter=';')
landen = read_excel("extra gegevens/landen.xlsx", sheetname="uitgebreide lijst 2014")

Laten we even in detail deze tabellen bekijken. De landen:

In [7]:
landen.head()

,Continent (staatkundig),Land,Soort mention,Mention
0,Azië,Afghanistan,korte landnaam,Afghanistan
1,Azië,Afghanistan,inwoner,Afghaan
2,Azië,Afghanistan,inwoner,Afghanen
3,Azië,Afghanistan,adjectief,Afghaans
4,Azië,Afghanistan,adjectief,Afghaanse


Voor elk land weten we in welk (staatkundig) continent het ligt, en we hebben in de kolom 'Mention' verschillende manieren waarop dat land kan voorkomen in de tekst.

Bij typering zien we het volgende:

In [8]:
typering.tail()

,Gemeente Origineel,Postcode,Fusiegemeente,Stedelijkheid fusiegemeenten,Gemeente cluster,Province (English),Country (English),Provincie,Stedelijkheid
931,Kwaremont (Kluisbergen),NaN,Kluisbergen,Niet-stedelijk,Kluisbergen,East Flanders,Belgium,Oost-Vlaanderen,Niet-stedelijk
932,Kleine-Brogel (Peer),NaN,Peer,Niet-stedelijk,Peer,Limburg,Belgium,Limburg,Niet-stedelijk
933,Meilegem (Zwalm),NaN,Zwalm,Niet-stedelijk,Zwalm,East Flanders,Belgium,Oost-Vlaanderen,Niet-stedelijk
934,Beek (Bree),NaN,Bree,Niet-stedelijk,Bree,Limburg,Belgium,Limburg,Niet-stedelijk
935,Oetingen (Gooik),NaN,Gooik,Niet-stedelijk,Gooik,Flemish Brabant,Belgium,Vlaams-Brabant,Niet-stedelijk


De kolom Gemeente Origineel is de naam van de gemeente in de uitdatabank gegevens, en we kunnen de naam in de kolom Fusiegemeente en Province (English) gebruiken om een genormaliseerd zicht te krijgen.

Tot slot hebben we nog de coordinaten:

In [9]:
coord.head()

,Fusiegemeente,adress,latitude,longitude
0,Aalst,"Aalst, Belgium","50,9378101","4,0409517"
1,Aalter,"Aalter, Belgium","51,087349","3,448371"
2,Aarschot,"Aarschot, Belgium","50,9859959","4,8365218"
3,Aartselaar,"Aartselaar, Belgium","51,1340539","4,3844742"
4,Affligem,"Affligem, Belgium","50,9084","4,11281"


Hiermee kunnen we voor iedere Fusiegemeente (zie vorige tabel) de latitude en longitude ophalen.

Nu gaan we voor ieder event in de UiTdatabankgegevens na welk land er vermeld wordt in de beschrijving van dat event. We houden ook al onmiddellijk bij wat de genormaliseerde naam is van de gemeente en de coordinaten van het centrum. Bovendien tellen we binnen het land ook nog de verschillende disciplines.

In [10]:
data = {}
niet_vlaams = ["Jodoigne", "Tournai", "Escanaffles"]
count = 1
for row in df.iterrows():
    if count % 1000 == 0:
        print(count, "of", len(df.index))
    count += 1
    tekst = row[1]["Tekst"]
    gemeente = row[1]["Gemeente"]
    if str(gemeente) != "nan" and str(gemeente) not in niet_vlaams:
        for land in set(landen["Land"].values):
            for mention in landen[landen["Land"] == land]["Mention"]:
                regex = compile(r"\b" + mention + r"\b", IGNORECASE)
                amount_of_mentions = len(regex.findall(tekst))
                if amount_of_mentions > 0:
                    try:
                        typeringlijn = typering[typering["Gemeente Origineel"] == gemeente]
                        fusiegemeente = typeringlijn["Fusiegemeente"].values[0]
                        provincie = typeringlijn["Province (English)"].values[0]
                        lat = coord[coord["Fusiegemeente"] == fusiegemeente]["latitude"].values[0]
                        lon = coord[coord["Fusiegemeente"] == fusiegemeente]["longitude"].values[0]
                        continent = landen[landen["Mention"] == mention]["Continent (staatkundig)"].values[0]
                        mention_soort = landen[landen["Mention"] == mention]["Soort mention"].values[0]
                        discipline = row[1]["Discipline"]
                        subdiscipline = row[1]["Subdiscipline"]
                        if fusiegemeente not in data:
                            data[fusiegemeente] = {"latitude": float(lat.replace(',', '.')),
                                                   "longitude": float(lon.replace(',', '.')),
                                                   "provincie": provincie,
                                                   "vermeldingen": {}
                                                   }
                        if continent not in data[fusiegemeente]["vermeldingen"]:
                            data[fusiegemeente]["vermeldingen"][continent] = {}
                        if land not in data[fusiegemeente]["vermeldingen"][continent]:
                            data[fusiegemeente]["vermeldingen"][continent][land] = {}
                        if discipline not in data[fusiegemeente]["vermeldingen"][continent][land]:
                            data[fusiegemeente]["vermeldingen"][continent][land][discipline] = {}
                        if subdiscipline not in data[fusiegemeente]["vermeldingen"][continent][land][discipline] :
                            data[fusiegemeente]["vermeldingen"][continent][land][discipline][subdiscipline] = 0
                        data[fusiegemeente]["vermeldingen"][continent][land][discipline][subdiscipline] += amount_of_mentions
                    except:
                        print(gemeente)

1000 of 33028
2000 of 33028
3000 of 33028
4000 of 33028
5000 of 33028
6000 of 33028
7000 of 33028
8000 of 33028
9000 of 33028
10000 of 33028
11000 of 33028
12000 of 33028
13000 of 33028
14000 of 33028
15000 of 33028
16000 of 33028
17000 of 33028
18000 of 33028
19000 of 33028
20000 of 33028
21000 of 33028
22000 of 33028
23000 of 33028
24000 of 33028
25000 of 33028
26000 of 33028
27000 of 33028
28000 of 33028
29000 of 33028
30000 of 33028
31000 of 33028
Vaalbeek (Oud-Heverlee)
32000 of 33028
33000 of 33028


We kunnen al eens gauw de structuur van deze gegevens nakijken voor bijvoorbeeld de gemeente Brussel:

In [11]:
print(dumps(data["Lint"], indent=2))

{
  "vermeldingen": {
    "Azi\u00eb": {
      "Japan": {
        "podium": {
          "Theatervoorstelling": 1
        }
      },
      "India": {
        "Tentoonstelling": {
          "NaN": 4
        }
      }
    },
    "Europa": {
      "Frankrijk": {
        "podium": {
          "Theatervoorstelling": 2
        }
      },
      "Letland": {
        "podium": {
          "Theatervoorstelling": 2
        }
      },
      "Nederland": {
        "podium": {
          "Theatervoorstelling": 1
        }
      },
      "Duitsland": {
        "podium": {
          "Theatervoorstelling": 2
        }
      },
      "Vlaanderen": {
        "podium": {
          "Theatervoorstelling": 1
        }
      }
    }
  },
  "latitude": 51.1272719,
  "longitude": 4.4967604,
  "provincie": "Antwerp"
}


We kunnen nu deze gegevens samenballen in een tabel.

In [12]:
lines = []
for fusiegemeente in data:
    lat = data[fusiegemeente]["latitude"]
    lon = data[fusiegemeente]["longitude"]
    prov = data[fusiegemeente]["provincie"]
    for continent in data[fusiegemeente]["vermeldingen"]:
        for land in data[fusiegemeente]["vermeldingen"][continent]:
            for discipline in data[fusiegemeente]["vermeldingen"][continent][land]:
                for subdiscipline in data[fusiegemeente]["vermeldingen"][continent][land][discipline]:
                    lines.append([fusiegemeente, lat, lon, prov, continent, land, discipline, subdiscipline, 
                                  data[fusiegemeente]["vermeldingen"][continent][land][discipline][subdiscipline]])
res = DataFrame(lines, columns=['Gemeente', 'Latitude', 'Longitude', 'Provincie', 'Continent', 'Land', 'Discipline', 'Subdiscipline', 'Frequentie'])

De tabel bevat dus de volgende informatie:

In [20]:
res.head()

,Gemeente,Latitude,Longitude,Provincie,Continent,Land,Discipline,Subdiscipline,Frequentie
0,Langemark-Poelkapelle,50.913567,2.920013,West Flanders,Europa,Frankrijk,Tentoonstelling,NaN,3
1,Langemark-Poelkapelle,50.913567,2.920013,West Flanders,Europa,Frankrijk,podium,Theatervoorstelling,2
2,Langemark-Poelkapelle,50.913567,2.920013,West Flanders,Europa,Italië,podium,Theatervoorstelling,13
3,Langemark-Poelkapelle,50.913567,2.920013,West Flanders,Europa,Letland,podium,Theatervoorstelling,5
4,Langemark-Poelkapelle,50.913567,2.920013,West Flanders,Europa,Vlaanderen,Tentoonstelling,NaN,4


Als voorbeeld kan de eerste lijn gelezen worden als "in de gemeente Denderleeuw (coordinaten 50.88, 4.07) wordt India 1 keer vermeld".

We schrijven tot slot deze tabel uit naar een CSV bestand voor verdere analyse.

In [14]:
res.to_csv("landsvermeldingen.csv")